# 01. Import libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [2]:
# Create a path for easier importing
path = r'C:\Users\blim9\Desktop\Projects'

In [3]:
df = pd.read_csv(os.path.join (path, 'Netflix Data Analysis', 'Data', 'netflix_titles.csv'), index_col=False)

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


#### Based on the shape, there are 8807 rows and 12 columns. The above info() shows that are several columns with null values.

# 02. Data wrangling

In [6]:
# Check the types of content Netflix offers
df.type.value_counts()

Movie      6131
TV Show    2676
Name: type, dtype: int64

In [7]:
# Check the ratings of the content
df.rating.value_counts()

TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: rating, dtype: int64

In [8]:
# Duration cannot be a rating, so it will be removed since there are only 3 values
df = df[df["rating"].str.contains("min")==False]

In [9]:
df.rating.value_counts()

TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
Name: rating, dtype: int64

In [10]:
# Check for missing values
df.isnull().sum()

show_id            0
type               0
title              0
director        2631
cast             825
country          830
date_added        10
release_year       0
rating             0
duration           0
listed_in          0
description        0
dtype: int64

#### There are many missing values for the director column as well as the cast column, but I don't think they will be very relevant.
#### There are also 10 missing values for date_added which can also be removed as it will not impact the data set significantly.

In [11]:
# Drop the director and cast columns
df = df.drop(columns = ['director','cast'])

In [12]:
df.dropna(subset = ['date_added'], inplace=True)

In [13]:
df.isnull().sum()

show_id           0
type              0
title             0
country         829
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

#### Country will be an important column to generate a map, so I will use the mode function to fill it in.

In [14]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

In [15]:
# Create new columns based on the date columns (separate by year and month)
# Start by extracting the year from date_added
df['year_added'] = df['date_added'].apply(lambda x: x.split(" ")[-1])
df['year_added'].head()

0    2021
1    2021
2    2021
3    2021
4    2021
Name: year_added, dtype: object

In [16]:
# Extract the month from date_added
df['month_added'] = df['date_added'].apply(lambda x: x.split(" ")[0])
df['month_added'].head()

0    September
1    September
2    September
3    September
4    September
Name: month_added, dtype: object

In [17]:
df.country.value_counts()

United States                             3638
India                                      972
United Kingdom                             418
Japan                                      243
South Korea                                199
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: country, Length: 748, dtype: int64

In [18]:
# Apply the same lambda function on the country column as there are multiple values
# Create a new column with just the first country
df['original_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['original_country'].head()

0    United States
1     South Africa
2    United States
3    United States
4            India
Name: original_country, dtype: object

In [19]:
# Create a new dataframe for visualizations
df_split = df.copy()

In [20]:
df_split = pd.concat([df_split, df["listed_in"].str.split(",", expand = True)], axis = 1)
df_split

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,original_country,0,1,2
0,s1,Movie,Dick Johnson Is Dead,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021,September,United States,Documentaries,None,None
1,s2,TV Show,Blood & Water,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021,September,South Africa,International TV Shows,TV Dramas,TV Mysteries
2,s3,TV Show,Ganglands,United States,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021,September,United States,Crime TV Shows,International TV Shows,TV Action & Adventure
3,s4,TV Show,Jailbirds New Orleans,United States,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021,September,United States,Docuseries,Reality TV,None
4,s5,TV Show,Kota Factory,India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021,September,India,International TV Shows,Romantic TV Shows,TV Comedies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",2019,November,United States,Cult Movies,Dramas,Thrillers
8803,s8804,TV Show,Zombie Dumb,United States,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",2019,July,United States,Kids' TV,Korean TV Shows,TV Comedies
8804,s8805,Movie,Zombieland,United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,2019,November,United States,Comedies,Horror Movies,None
8805,s8806,Movie,Zoom,United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",2020,January,United States,Children & Family Movies,Comedies,None


In [21]:
df_split = df_split.melt(id_vars = ["type", "title"], value_vars = range(3), value_name = "genre")
df_split = df_split[df_split["genre"].notna()]
df_split

,type,title,variable,genre
0,Movie,Dick Johnson Is Dead,0,Documentaries
1,TV Show,Blood & Water,0,International TV Shows
2,TV Show,Ganglands,0,Crime TV Shows
3,TV Show,Jailbirds New Orleans,0,Docuseries
4,TV Show,Kota Factory,0,International TV Shows
...,...,...,...,...
26363,TV Show,Zindagi Gulzar Hai,2,TV Dramas
26364,Movie,Zinzana,2,Thrillers
26365,Movie,Zodiac,2,Thrillers
26366,TV Show,Zombie Dumb,2,TV Comedies


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8790 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   show_id           8790 non-null   object
 1   type              8790 non-null   object
 2   title             8790 non-null   object
 3   country           8790 non-null   object
 4   date_added        8790 non-null   object
 5   release_year      8790 non-null   int64 
 6   rating            8790 non-null   object
 7   duration          8790 non-null   object
 8   listed_in         8790 non-null   object
 9   description       8790 non-null   object
 10  year_added        8790 non-null   object
 11  month_added       8790 non-null   object
 12  original_country  8790 non-null   object
dtypes: int64(1), object(12)
memory usage: 961.4+ KB


In [23]:
# Change the year_added column into int16
df.year_added = df.year_added.astype('int16')

In [24]:
# Change the release_year column into int16
df.release_year= df.release_year.astype('int16')

In [25]:
df.head()

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,original_country
0,s1,Movie,Dick Johnson Is Dead,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021,September,United States
1,s2,TV Show,Blood & Water,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021,September,South Africa
2,s3,TV Show,Ganglands,United States,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021,September,United States
3,s4,TV Show,Jailbirds New Orleans,United States,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021,September,United States
4,s5,TV Show,Kota Factory,India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021,September,India


# 03. Data consistency check

In [26]:
df.isnull().sum()

show_id             0
type                0
title               0
country             0
date_added          0
release_year        0
rating              0
duration            0
listed_in           0
description         0
year_added          0
month_added         0
original_country    0
dtype: int64

### No more missing values

In [27]:
# Check for duplicate values
df_dups = df[df.duplicated()]

In [28]:
df_dups

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,original_country


### No duplicates

In [29]:
# Export the cleaned data set for visualization
df.to_csv(os.path.join (path, 'Netflix Data Analysis', 'Data', 'cleaned_netflix.csv'))

In [30]:
df_split.to_csv(os.path.join (path, 'Netflix Data Analysis', 'Data', 'split_data.csv'))